In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer
import evaluate
## nltk для бейзлайновой модели
import nltk
from nltk.tokenize import sent_tokenize
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer

/home/gallirium/dostNN/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = 'cointegrated/rut5-small'
dataset = load_dataset('IlyaGusev/gazeta', revision="v2.0")
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [3]:
inputs = tokenizer("Это вообще что такое??")

In [4]:
tokenizer.convert_ids_to_tokens(inputs.input_ids)

['▁', 'Это', '▁в', 'ообще', '▁что', '▁тако', 'е', '??', '</s>']

In [5]:
def process(dset):
    model_inputs = tokenizer(dset['text'], max_length=512, truncation=True)
    labels = tokenizer(dset['summary'], max_length=50, truncation=True)
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [6]:
tok_dset = dataset.map(process,batched=True)
rouge = evaluate.load("rouge")

In [7]:
def three_sentences(text):
    return '\n'.join(sent_tokenize(text)[:3])

In [8]:
three_sentences(dataset['train'][1]['text'])

'Словосочетание «музыкальный кинофестиваль» уже не звучит непривычным оксюмороном — в наше бедное на гениев время разобраться в современной музыке порой проще с помощью наблюдения за участниками процесса.\nТак уже было, к примеру, с сандэнсовским фильмом «DiG!», влюбившим зрителей по всему миру в портлендского безумца Антона Ньюкомба и его группу Brian Jonestown Massacre.\nВ столице картину показали год назад.'

In [9]:
dataset['train'][1]['text']

"Словосочетание «музыкальный кинофестиваль» уже не звучит непривычным оксюмороном — в наше бедное на гениев время разобраться в современной музыке порой проще с помощью наблюдения за участниками процесса. Так уже было, к примеру, с сандэнсовским фильмом «DiG!», влюбившим зрителей по всему миру в портлендского безумца Антона Ньюкомба и его группу Brian Jonestown Massacre. В столице картину показали год назад. Это был совсем камерный клубный показ в рамках фестиваля Music.doc, почти без рекламы, но о тех двух часах не пожалел никто из пары десятков случайных и не очень посетителей. То же можно сказать и о прочих показах. Стартующий в этот четверг Beat Film Festival – прямой наследник Music.doc. На этот раз к услугам киномеломанов самая авторитетная арт-хаусная площадка города, а не тесный клуб или помпезный «Иллюзион», а вместо собранной по сусекам программы – фильмы с не более чем двухлетним сроком давности. Картины «Bassweight» и «Favellla on Blast» рассказывают (соответственно) о все 

In [10]:
## baseline three-sentence summarization
summaries = [three_sentences(text) for text in dataset['train'][:]['text']]
results = rouge.compute(predictions=summaries, references=dataset['train'][:]['summary'])

In [11]:
results = {k: round(v*100,2) for k, v in results.items()}

In [12]:
results

{'rouge1': 17.45, 'rouge2': 5.24, 'rougeL': 16.89, 'rougeLsum': 16.93}

In [13]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [14]:
import numpy as np

def metrics(p):
    preds, labels = p
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch.decode(labels, skip_special_tokens=True)

    decoded_preds = ['\n'.join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ['\n'.join(sent_tokenize(pred.strip())) for label in decoded_labels]

    result = rouge_score.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return {k: round(v*100, 2) for k, v in result.items()}

In [15]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

tok_dset_test = tok_dset.remove_columns(['text','summary','title','url','date'])

In [16]:
BS = 8
EPOCHS = 2
LOGS = len(tok_dset_test['train'])//BS

train_args = Seq2SeqTrainingArguments(
    output_dir="rut5-base-summarization-practice",
    evaluation_strategy='epoch',
    learning_rate=5e-4,
    per_device_train_batch_size=BS,
    per_device_eval_batch_size=BS,
    weight_decay=0.02,
    num_train_epochs=EPOCHS,
    logging_steps=LOGS,
    predict_with_generate=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args = train_args,
    train_dataset=tok_dset_test["train"],
    eval_dataset=tok_dset_test["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=metrics,
)

In [17]:
trainer.train()

Epoch,Training Loss,Validation Loss


/home/gallirium/dostNN/lib/python3.11/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


AttributeError: 'T5Tokenizer' object has no attribute 'batch'

In [ ]:
trainer_evaluate()